In [ ]:
## 合并视频文件
import os
import subprocess

# 示例调用
source_directory = r"C:\Users\GMK_M7\Videos\bili\113043002360725"  # 源目录
target_directory = r"C:\Users\GMK_M7\Videos\temp\从零开始学Scrapy爬虫框架"  # 目标目录

def contains_m4s_file(directory_path):
    # 检查路径是否存在且是目录
    if not os.path.exists(directory_path) or not os.path.isdir(directory_path):
        print(f"The path {directory_path} does not exist or is not a directory.")
        return False
    
    # 列出指定目录下的所有文件和子目录
    for entry in os.listdir(directory_path):
        # 构建完整的路径
        full_path = os.path.join(directory_path, entry)
        
        # 检查是文件且以.m4s为后缀
        if os.path.isfile(full_path) and entry.endswith('.m4s'):
            return True
    
    return False

def list_directory_contents(path):
    # 检查路径是否存在且是目录
    if not os.path.exists(path) or not os.path.isdir(path):
        print(f"The path {path} does not exist or is not a directory.")
        return
    
    # 列出指定目录下的所有文件和子目录
    for entry in os.listdir(path):
        # 构建完整的路径
        full_path = os.path.join(path, entry)
        
        # 检查是文件还是目录
        if os.path.isdir(full_path):
            print(f"Directory: {full_path}")
            exist = contains_m4s_file(full_path)
            if(exist):
                print(f'存在m4s文件')
                # command_text = r"ffmpeg -i {path}\video.m4s -i {path}\audio.m4s -codec copy {path}\output.mp4".format(path=full_path)
                # 构建 ffmpeg 命令
                command = [
                    'ffmpeg',
                    '-i', '{}\\video.m4s'.format(full_path),  # 输入视频文件
                    '-i', '{}\\audio.m4s'.format(full_path),  # 输入音频文件
                    '-codec', 'copy',  # 直接复制流，不重新编码
                    '-y', # 强制覆盖输出文件
                    '{}\\output.mp4'.format(full_path)  # 输出文件
                ]
                print(f'command: ->>> {command}')
                process  = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                # 等待命令执行完成
                stdout, stderr = process.communicate()
                # 检查命令是否成功执行
                if process.returncode == 0:
                    print(f"合并成功！输出文件: {full_path}\\output.mp4")
                else:
                    print(f"合并失败！错误信息:\n{stderr.decode('utf-8')}")

            else:
                list_directory_contents(full_path)

        elif os.path.isfile(full_path):
            print(f"File: {full_path}")

list_directory_contents(source_directory)

In [ ]:
## 重命名
import json
# import time
# import datetime
from datetime import datetime

def contains_entry_json(directory_path):
    # 检查路径是否存在且是目录
    if not os.path.exists(directory_path) or not os.path.isdir(directory_path):
        print(f"The path {directory_path} does not exist or is not a directory.")
        return False
    
    # 构建entry.json的完整路径
    entry_json_path = os.path.join(directory_path, 'entry.json')
    
    # 检查entry.json是否存在且是文件
    if os.path.isfile(entry_json_path):
        return True
    else:
        return False
    
def list_directory_contents(path):
    # 检查路径是否存在且是目录
    if not os.path.exists(path) or not os.path.isdir(path):
        print(f"The path {path} does not exist or is not a directory.")
        return
        # 列出指定目录下的所有文件和子目录
    for entry in os.listdir(path):
        # 构建完整的路径
        full_path = os.path.join(path, entry)
        
        # 检查是文件还是目录
        if os.path.isdir(full_path):
            print(f"Directory: {full_path}")
            exist = contains_entry_json(full_path)
            if(exist):
                entry_json_path = os.path.join(full_path, 'entry.json')
                # 读取 JSON 文件
                with open(entry_json_path, "r", encoding="utf-8") as file:
                    data = json.load(file)
                page_data = data["page_data"]

                #time_update_stamp = data["time_update_stamp"][:-3] # 读取文件时间戳，转年月日。时间戳太大了，非法参数，去掉后三位的毫秒。int不能切片，直接除
                time_update_stamp = data["time_update_stamp"]//1000
                print(f"{time_update_stamp}")
                dateArray = datetime.fromtimestamp(time_update_stamp)
                time_update = dateArray.strftime("%Y-%m-%d_%H_%M_%S")
                
                part_title = page_data["part"].replace("/","_").replace("\\","_").replace("|","_").replace("*","_").replace("?","_").replace(":","_").replace("\"","_").replace(">","_").replace("<","_")
                print(f'{part_title}_{time_update}')

                # 子目录要么80,要么64, 获取当前目录
                # 列出当前目录下的所有内容
                contents = os.listdir(full_path)
                # 筛选出子目录
                subdirs = [item for item in contents if os.path.isdir(os.path.join(full_path, item))]
                if subdirs:
                    first_subdir = subdirs[0]

                    
                    mp4_path= os.path.join(full_path, first_subdir,'output.mp4')
                    mp4_new_path = os.path.join(full_path, first_subdir,'{}.mp4'.format(part_title+"_"+time_update))
                    if os.path.isfile(mp4_path):
                        print(f'重命名:{mp4_path},->,{mp4_new_path}')
                        os.rename(mp4_path, mp4_new_path)
                    else:
                        print(f'无文件存在:{mp4_path}')
            else:
                list_directory_contents(full_path)
            
        elif os.path.isfile(full_path):
            # print(f"File: {full_path}")
            pass

list_directory_contents(source_directory)

In [ ]:
## 移动到某个目录

import shutil

def move_mp4_files(source_dir, target_dir):
    """
    将 source_dir 目录下所有 .MP4 文件移动到 target_dir 目录。
    
    :param source_dir: 源目录路径
    :param target_dir: 目标目录路径
    """
    # 确保目标目录存在，如果不存在则创建
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # 遍历源目录下的所有文件和子目录
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            # 检查文件是否以 .MP4 结尾（不区分大小写）
            if file.lower().endswith('.mp4'):
                # 源文件的完整路径
                source_file_path = os.path.join(root, file)
                # 目标文件的完整路径
                target_file_path = os.path.join(target_dir, file)

                # 如果目标文件已存在，则跳过或重命名
                if os.path.exists(target_file_path):
                    print(f"文件已存在，跳过: {target_file_path}")
                    continue

                # 移动文件
                shutil.move(source_file_path, target_file_path)
                print(f"已移动: {source_file_path} -> {target_file_path}")
        
        for dir in dirs:
            move_mp4_files(dir,target_dir)


move_mp4_files(source_directory, target_directory)